While there are records of instances of school shootings in the United States dating back to the Enoch Brown school massacre in Greencastle, PA on July 26, 1764, the first mass school shooting according to the modern convention of counting a mass school shooting as being a three person or more causualty on the actual school grounds did not occur until March 26, 1893 in Plain Dealing, LA during an evening dance at the school.  According to Wikipedia beween then and 1966 only three other mass school shottings occured.  They were as follows:
* The December 13, 1898 Charleston, WV school exhibition incident killing 6 and wounding at least 1 other
* The May 6, 1940 Pasadena, CA - South Pasadena Junior High School - murders leaving 5 dead and 2 others wounded
* The February 2, 1960 Hartford City, IN - William Reed School - shooting killing 3
This means that only four modern day standard mass school shootings occured before 1966 even though around 130 school-related shootings are on record with Wikipedia from this early time period in America history, they not meeting the modern definition of being an actual mass school shooting by today's standards.

In 1968 & 1970, a total of 9 student protesters were killed and another 48 were injured at the hands of United States government officers.  These three incidents included the disgraceful South Carolina State University racist protesters in Orangeburg, SC on February 8, 1968, the Kent State University Vietnam War protesters by armed National Guard solders on May 4, 1970 in Kent, OH, and police shot the protestors at the Jackson State University United States military presence in Cambodia on May 15, 1970 (only eleven days following the Kent State massacre) in Jackson, MS.

While, the Columbine High School massacre did not happen until 1999, these attacks by the United States government and the August 1, 1966 University of Texas - Austin massacre in Austin, Texas killing 17 and wounding 31 paired with the November 12, 1966 (roughly 3.5 months after the University of Texas massacre) in Mesa, AZ where 5 died and another 2 were injured, set off a new type of shock value the vast number killed in a particular incident.

Now, in the first month and a half of 2018, the United States not only has had the deadliest mass school shooting in United States history, but has had a school shooting rate increase to 1 occuring in every 60 hours.  Many National Rifle Association backed politicans continue to look the other way or actively try to stawl any and all gun-control reform or legislation.

While it is clear that gun violence is not possible without guns, the interpretation of the "Second Amendment" of "The Bill of Rights" to the "United States Constitution" protects at least some of her citizens to bare arms under certain well-regulated circumstances.  Which there in this the Founders brilliance lies both the pearl of wisdom and the rub.  It is clear from the wording of this ammendment verses all the others that the Founders signing the Constitution clearly intended it to have breathing room in case of a foreign invasion, civil war, hunting, or any other communal benefit as escribed by the federal government as it is their most basic duty to The People to pretect and defend the country and all her citizens.

Having said that, I believe we as a society have run a stray regarding gun mentality.  In the days of the "Wild West," salon and high noon shootouts occurred frequently.  So, did bandits, robbers, and criminal behaviors we have tried desparately to outlaw ever since -- sometimes to what appears no to little avail.

Like any Python project, it is always best practice to put all your import libraries at the top of a file both for easy access and documenting reference; as well as, for the functional purpose of ensuring the file has properly loaded the libraries prior to their need within the program itself.

In [1]:
# Import Libraries

from csv import DictReader
from io import BytesIO
from sqlite3 import connect, Error, version
from urllib.request import build_opener, urlopen
from zipfile import ZipFile
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
from openpyxl import load_workbook
import bokeh
import numpy

Next, let's retrieve the datasets necessary for the project via the internet.

In [2]:
# Unzip Stanford MSA Database CSV file

msa_url = urlopen("https://stanford.box.com/shared/static/75lmj9rvw9mhtfqfjijhcc6oa15o5m4a.zip")
zipfile = ZipFile(BytesIO(msa_url.read()))
zipfile.namelist()

['MSA/',
 'MSA/Stanford_MSA_Data_Dictionary (V.5).pdf',
 '__MACOSX/',
 '__MACOSX/MSA/',
 '__MACOSX/MSA/._Stanford_MSA_Data_Dictionary (V.5).pdf',
 'MSA/Stanford_MSA_Database_for_release_06142016.xlsx',
 '__MACOSX/MSA/._Stanford_MSA_Database_for_release_06142016.xlsx']

In [3]:
# Load Workbook & Proper Worksheet

wb = load_workbook(zipfile.extract('MSA/Stanford_MSA_Database_for_release_06142016.xlsx','store', pwd=None))
ws = wb['Published']
print(ws)

<Worksheet "Published">


In [4]:
# Convert Excel file to Comma Separated Values file

msa_xls = pd.read_excel('store/MSA/Stanford_MSA_Database_for_release_06142016.xlsx', 'Published', index_col=None)
msa_xls.to_csv('store/MSA/Stanford_MSA_Database_for_release.csv', encoding='utf-8', index=False)

In [10]:
# Create a SQL database connection

conn = connect('temp/truth.sql')
print(version)

conn.text_factory = str  # allows utf-8 data to be stored

c = conn.cursor()

tablename = "Shootings"

with open('store/MSA/Stanford_MSA_Database_for_release.csv', newline='') as csvfile:
    msa_reader = DictReader(csvfile, delimiter=",", dialect='excel', quotechar='"')

    header = True
    for row in msa_reader:
        try:
            if header:
                # gather column names from the first row of the csv
                header = False

                sql = "DROP TABLE IF EXISTS %s" % tablename
                c.execute(sql)

                column_name_correction = []
                print([column_name_correction.append(column.replace(' ', '_') for column in row )])
                # print(", ".join([ "%s text" % column for column in column_name_correction ]))
                sql = "CREATE TABLE %s (%s)" % (tablename,
                                                ", ".join([ "%s text" % column for column in column_name_correction ]))
                c.execute(sql)
                # print(c.execute(sql))

                for column in row:
                    if column.lower().endswith("_id"):
                        index = "%s__%s" % ( tablename, column )
                        sql = "CREATE INDEX %s on %s (%s)" % ( index, tablename, column )
                        c.execute(sql)
                        # print(c.execute(sql))

            insertsql = "INSERT INTO %s VALUES (%s)" % (tablename,
                                                            ", ".join([ "?" for column in row ]))
            # print(insertsql)

            rowlen = len(row)
            # else:
            #     # skip lines that don't have the right number of columns
            #     if len(row) == rowlen:
            #         c.execute(insertsql, row)
        except Error as error:
            exit('file {}, line {}: {}'.format(csvfile, msa_reader.line_num, error))
            print(error)

    conn.commit()

2.6.0
[None]
near "<": syntax error


In [6]:
# File for Web Scrapping & Parsing

gun_legislation_url = 'https://www.washingtonpost.com/graphics/national/gun-legislation/'
gun_legislation_html_soup = BeautifulSoup(urlopen(gun_legislation_url), 'html.parser')
stats_json_string = gun_legislation_html_soup.find('script', attrs={'src': 'js/base.js?c=a90e33a675e9d0ac31328c0ce0d2e12b16035540'})
print(stats_json_string)

<script charset="utf-8" src="js/base.js?c=a90e33a675e9d0ac31328c0ce0d2e12b16035540" type="text/javascript"></script>


In [7]:
full_gun_legislation_url = 'https://www.washingtonpost.com/graphics/national/gun-legislation/js/base.js?c=a90e33a675e9d0ac31328c0ce0d2e12b16035540'
legislation_request = requests.get(full_gun_legislation_url)

try:
    # print(legislation_request.headers)
    b = legislation_request.text.find("{first:")
    e = legislation_request.text.find("{}],2:")
    scrape = legislation_request.text[b - 1:e - 2]

    scrape = scrape.replace('first:', '"first":')
    scrape = scrape.replace('last:', '"last":')
    scrape = scrape.replace('fullLast:', '"fullLast":')
    scrape = scrape.replace('party:', '"party":')
    scrape = scrape.replace('house:', '"house":')
    scrape = scrape.replace('state:', '"state":')
    scrape = scrape.replace('control:', '"control":')
    scrape = scrape.replace('rights:', '"rights":')
    scrape = scrape.replace('id:', '"id":')
    scrape = scrape.replace('score:', '"score":')
    scrape = scrape.replace('pi:', '"pi":')
    scrape = scrape.replace('grade:', '"grade":')
    scrape = scrape.replace('gi:', '"gi":')

    scrape = json.dumps(scrape)
    print(scrape)

except Error as error:
    legislation_request.status_code

In [8]:
# Close the SQL connection to our SQLite database

c.close()
conn.close()

In [ ]:
# Libby MacKenzie
# Dash Kennedy
